## Process the initial data

In [ ]:
import numpy as np
import os, copy
import pandas as pd

from scipy import stats
from statsmodels.tsa.stattools import acf
from scipy import optimize
import copy

# Import data: extract data from database
from google.colab import drive
# this will prompt for authorization.
drive.mount('/content/drive')

path = '/content/drive/My Drive/personal_files/deeptop/'

In [ ]:
def load_all_file_path(dir_path):
    """
    输入数据文件夹
    :return: 每个csv文件的绝对路径
    """
    all_list_first = os.listdir(dir_path)
    detail_path = []
    for i in range(len(all_list_first)):
        if all_list_first[i].split('.')[-1] == 'csv':
            dir_second = os.path.join(dir_path, all_list_first[i])
            detail_path.append(dir_second)
    return detail_path


def seq_interpolation(signal):
    ori_hr = copy.deepcopy(signal)  # 原始hr序列，不做处理
    first_index = np.where(ori_hr > 0)[0][0]
    if ori_hr[0] == 0:
        ori_hr[:first_index] = signal[signal > 0][0]  # 初始为0，所有开始的0用后面的插
    null_index = np.where(ori_hr == 0)[0]
    for i in null_index:
        ori_hr[i] = ori_hr[i - 1]
    return ori_hr

## Positive data  
including id info

In [ ]:
if __name__ == '__main__':
    pos_data_dir = path + 'data/clinical_event_data/posdata/'
    all_pos_path_list = load_all_file_path(pos_data_dir)
    used_data_len = 2 * 60  # 使用两小时数据
    first_used = 12  # 第几小时开始取
    all_stacked_pos = np.array([])
    for ind, v in enumerate(all_pos_path_list):
        # print(ind)
        temp_array = np.array([])
        temp_f = pd.read_csv(v)
        temp_f = temp_f.fillna(0)
        event_label = temp_f['HR_trachy_event'].values
        # num_pos_event = len(event_label[event_label == 1])
        # for i in range(num_pos_event):
        i = 0
        current_start, current_end = i + first_used*60, i + used_data_len + first_used*60
        used_hr = temp_f['HR'].values[current_start:current_end]
        if len(used_hr[used_hr == 0]) > used_data_len * 0.3:
            continue
        inter_hr = seq_interpolation(used_hr)
        temp_array = inter_hr
        used_sys = temp_f['sys'].values[current_start:current_end]
        if len(used_hr[used_sys == 0]) > used_data_len * 0.3:
            continue
        inter_sys = seq_interpolation(used_sys)
        temp_array = np.vstack((temp_array, inter_sys))
        used_mean = temp_f['mean'].values[current_start:current_end]
        if len(used_hr[used_mean == 0]) > used_data_len * 0.3:
            continue
        inter_mean = seq_interpolation(used_mean)
        temp_array = np.vstack((temp_array, inter_mean))
        used_resp = temp_f['resp'].values[current_start:current_end]
        if len(used_hr[used_resp == 0]) > used_data_len * 0.3:
            continue
        inter_resp = seq_interpolation(used_resp)
        temp_array = np.vstack((temp_array, inter_resp))
        used_ox = temp_f['spo2'].values[current_start:current_end]
        if len(used_hr[used_ox == 0]) > used_data_len * 0.3:
            continue
        inter_ox = seq_interpolation(used_ox)
        # print(v)
        temp_array = np.vstack((temp_array, inter_ox))

        # add id info
        location_info = []
        location_info = [i for i, x in enumerate(all_pos_path_list[0]) if x == "/"]
        id = int(v[location_info[-1]+2:v.find('-')])

        temp_array = np.hstack((temp_array, [[1,id], [1,id], [1,id], [1,id], [1,id]]))
        if len(all_stacked_pos) == 0:
            all_stacked_pos = temp_array
        else:
            all_stacked_pos = np.vstack((all_stacked_pos, temp_array))
    all_stacked_pos = np.reshape(all_stacked_pos, (int(all_stacked_pos.shape[0] / 5), 5, 122))
    np.savez(path + 'data/stage_1/' + 'aa_all_stacked_pos_0.npz', all_stacked_pos)
    print(all_stacked_pos.shape)

## Negative data

In [ ]:
if __name__ == '__main__':
    pos_data_dir = path + 'data/clinical_event_data/negdata_part/'
    all_pos_path_list = load_all_file_path(pos_data_dir)
    # all_pos_path_list = random.sample(all_pos_path_list, 30000)
    used_data_len = 2 * 60  # 前两小时
    all_stacked_pos = np.array([])
    for ind, v in enumerate(all_pos_path_list):
        # print(ind)
        temp_array = np.array([])
        temp_f = pd.read_csv(v)
        temp_f = temp_f.fillna(0)
        used_hr = temp_f['HR'].values[:used_data_len]
        if len(used_hr[used_hr == 0]) > used_data_len * 0.3:
            continue
        inter_hr = seq_interpolation(used_hr)
        temp_array = inter_hr
        used_sys = temp_f['sys'].values[:used_data_len]
        if len(used_hr[used_sys == 0]) > used_data_len * 0.3:
            continue
        inter_sys = seq_interpolation(used_sys)
        temp_array = np.vstack((temp_array, inter_sys))
        used_mean = temp_f['mean'].values[:used_data_len]
        if len(used_hr[used_mean == 0]) > used_data_len * 0.3:
            continue
        inter_mean = seq_interpolation(used_mean)
        temp_array = np.vstack((temp_array, inter_mean))
        used_resp = temp_f['resp'].values[:used_data_len]
        if len(used_hr[used_resp == 0]) > used_data_len * 0.3:
            continue
        inter_resp = seq_interpolation(used_resp)
        temp_array = np.vstack((temp_array, inter_resp))
        used_ox = temp_f['spo2'].values[:used_data_len]
        if len(used_hr[used_ox == 0]) > used_data_len * 0.3:
            continue
        inter_ox = seq_interpolation(used_ox)
        # print(v)
        temp_array = np.vstack((temp_array, inter_ox))
        # add id info
        location_info = []
        location_info = [i for i, x in enumerate(all_pos_path_list[0]) if x == "/"]
        id = int(v[location_info[-1]+2:v.find('-')])
        temp_array = np.hstack((temp_array, [[0,id], [0,id], [0,id], [0,id], [0,id]]))        
        # temp_array = np.hstack((temp_array, [[0], [0], [0], [0], [0]]))
        if len(all_stacked_pos) == 0:
            all_stacked_pos = temp_array
        else:
            all_stacked_pos = np.vstack((all_stacked_pos, temp_array))
    all_stacked_pos = np.reshape(all_stacked_pos, (int(all_stacked_pos.shape[0] / 5), 5, 122))
    np.savez(path + 'data/stage_1/' + 'aa_all_stacked_neg.npz', all_stacked_pos)
    print(all_stacked_pos.shape)

## Create statistic features

In [ ]:
def cal_mean(signal):
    return np.mean(signal)


def cal_sd(signal):
    return np.std(signal)


def agg_autocorrelation(x):
    var = np.var(x)
    n = len(x)
    if np.abs(var) < 10**-10 or n == 1:
        a = 0
    else:
        a = acf(x, unbiased=True, fft=n > 1250)[1:]
    return a


def f_1(x, A, B):
    return A*x + B


def first_order_reg(signal):
    x0 = np.arange(0, len(signal))
    a1, _ = optimize.curve_fit(f_1, x0, signal)[0]
    return a1


def abs_energy(x):

    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)


def c3(x, lag):

    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    n = x.size
    if 2 * lag >= n:
        return 0
    else:
        return np.mean((np.roll(x, 2 * -lag) * np.roll(x, -lag) * x)[0:(n - 2 * lag)])

def seq_interpolation(signal):
    ori_hr = copy.deepcopy(signal)  # 原始hr序列，不做处理
    first_index = np.where(ori_hr > 0)[0][0]
    if ori_hr[0] == 0:
        ori_hr[:first_index] = signal[signal > 0][0]  # 初始为0，所有开始的0用后面的插
    null_index = np.where(ori_hr == 0)[0]
    for i in null_index:
        ori_hr[i] = ori_hr[i - 1]
    return ori_hr


def approximate_entropy(x, m, r):

    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)

    N = x.size
    r *= np.std(x)
    if r < 0:
        raise ValueError("Parameter r must be positive.")
    if N <= m+1:
        return 0

    def _phi(m):
        x_re = np.array([x[i:i+m] for i in range(N - m + 1)])
        C = np.sum(np.max(np.abs(x_re[:, np.newaxis] - x_re[np.newaxis, :]),
                          axis=2) <= r, axis=0) / (N-m+1)
        return np.sum(np.log(C)) / (N - m + 1.0)

    return np.abs(_phi(m) - _phi(m + 1))


def quantile(x, q):
    """
    Calculates the q quantile of x. This is the value of x greater than q% of the ordered values from x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param q: the quantile to calculate
    :type q: float
    :return: the value of this feature
    :return type: float
    """
    x = pd.Series(x)
    return pd.Series.quantile(x, q)


def sum_values(x):
    """
    Calculates the sum over the time series values

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: bool
    """
    if len(x) == 0:
        return 0

    return np.sum(x)


def mean_agg_auto(dfx):
    aggList=[]
    n = len(dfx.keys())
    def agg_autocorrelation1(i):
        var = np.var(i)
        n = len(i)
        if np.abs(var) < 10 ** -10 or n == 1:
            a = 0
        else:
            a = acf(i, unbiased=True, fft=n > 1250)[1:]
        return np.var(a)
    for x in dfx.keys():
        aggList.append(agg_autocorrelation1(pd.Series(dfx[x])))
    return sum(aggList)/n


def feature_extraction(signal):
    """
    :param signal:  5行 20列
    :return:
    """
    # x_pos = np.load('D:\\clinical_2\\aa_all_stacked_pos_12.npz')
    # x_pos = x_pos['arr_0'][:, :, :120]
    x_pos = signal
    num_features = 31
    current_fea = np.zeros(num_features)
    # 原始波形
    hr_data = x_pos[0, :]
    # hr_data = seq_interpolation(hr_data)
    res_data = x_pos[3, :]
    # hr_data = seq_interpolation(hr_data)
    ox_data = x_pos[4, :]
    # ox_data = seq_interpolation(ox_data)
    sys_data = x_pos[1, :]
    # sys_data = seq_interpolation(sys_data)
    dia_data = x_pos[2, :]
    # dia_data = seq_interpolation(dia_data)
    # HR
    hr_mean = cal_mean(hr_data)
    hr_sd = cal_sd(hr_data)
    hr_fir_reg = first_order_reg(hr_data)
    hr_abs_eng = abs_energy(hr_data)
    hr_c3 = c3(hr_data, 3)
    hr_c2 = c3(hr_data, 2)
    hr_q_07 = quantile(hr_data, 0.7)
    hr_q_01 = quantile(hr_data, 0.1)
    hr_q_03 = quantile(hr_data, 0.3)
    hr_sum = sum_values(hr_data)
    df = pd.DataFrame(np.transpose(signal))
    hr_mean_agg = mean_agg_auto(df)

    # RESP
    resp_mean = cal_mean(res_data)
    resp_sd = cal_sd(res_data)
    resp_fir_reg = first_order_reg(res_data)
    resp_abs_eng = abs_energy(res_data)
    resp_c3 = c3(res_data, 3)
    # OX
    ox_mean = cal_mean(ox_data)
    ox_sd = cal_sd(ox_data)
    ox_fir_reg = first_order_reg(ox_data)
    ox_abs_eng = abs_energy(ox_data)
    ox_c3 = c3(ox_data, 3)
    # SYS
    sys_mean = cal_mean(sys_data)
    sys_sd = cal_sd(sys_data)
    sys_fir_reg = first_order_reg(sys_data)
    sys_abs_eng = abs_energy(sys_data)
    sys_c3 = c3(sys_data, 3)
    # DIA
    dia_mean = cal_mean(dia_data)
    dia_sd = cal_sd(dia_data)
    dia_fir_reg = first_order_reg(dia_data)
    dia_abs_eng = abs_energy(dia_data)
    dia_c3 = c3(dia_data, 3)
    #
    current_fea[0] = hr_mean
    current_fea[1] = hr_sd
    current_fea[2] = hr_fir_reg
    current_fea[3] = hr_abs_eng
    current_fea[4] = hr_c3
    current_fea[5] = hr_c2
    current_fea[6] = hr_q_07
    current_fea[7] = hr_sum
    current_fea[8] = hr_mean_agg
    current_fea[9] = hr_q_01
    current_fea[10] = hr_q_03
    #
    current_fea[11] = resp_mean
    current_fea[12] = resp_sd
    current_fea[13] = resp_fir_reg
    current_fea[14] = resp_abs_eng
    current_fea[15] = resp_c3
    #
    current_fea[16] = ox_mean
    current_fea[17] = ox_sd
    current_fea[18] = ox_fir_reg
    current_fea[19] = ox_abs_eng
    current_fea[20] = ox_c3
    #
    current_fea[21] = sys_mean
    current_fea[22] = sys_sd
    current_fea[23] = sys_fir_reg
    current_fea[24] = sys_abs_eng
    current_fea[25] = sys_c3
    #
    current_fea[26] = dia_mean
    current_fea[27] = dia_sd
    current_fea[28] = dia_fir_reg
    current_fea[29] = dia_abs_eng
    current_fea[30] = dia_c3
    return current_fea

## Create positive samples  
categoried by four groups based on kmean

In [ ]:
cluster_id = pd.read_csv(path + 'data/cluster_id.csv')

In [ ]:
file_path = path + 'data/stage_1/' + 'aa_all_stacked_pos_6.npz' # [aa_all_stacked_pos_0,2,4,6]
x_neg = np.load(file_path)
x_neg = x_neg['arr_0']
x_neg_each, id_each = [], []
x_neg_each = x_neg
all_subjects, id_info_each = np.array([]), np.array([])
id_each = list(cluster_id.loc[cluster_id.flag == 1, 'subject_id'])
x_neg_each = x_neg_each[np.isin(x_neg_each[:, :, -1].astype(int), id_each),:].reshape((-1, 5, 122))
id_info_each = x_neg_each[:,:,-1] # patient id
x_neg_each = x_neg_each[:,:,:120]

In [ ]:
current_sub_signal = x_neg_each[0, :, :]

In [ ]:
current_sub_signal.shape

In [ ]:
windows_signal = current_sub_signal[:, 0*5:(0*5+20)]

In [ ]:
windows_signal.shape

In [ ]:
if __name__ == '__main__':
    path = '/content/drive/My Drive/personal_files/deeptop/'
    file_path = ''
    file_path = path + 'data/stage_1/' + 'aa_all_stacked_pos_6.npz' # [aa_all_stacked_pos_0,2,4,6]
    x_neg = np.load(file_path)
    x_neg = x_neg['arr_0']
    for m in range(4):
        x_neg_each, id_each = [], []
        x_neg_each = x_neg
        all_subjects, id_info_each = np.array([]), np.array([])
        id_each = list(cluster_id.loc[cluster_id.flag == m+1, 'subject_id'])
        x_neg_each = x_neg_each[np.isin(x_neg_each[:, :, -1].astype(int), id_each),:].reshape((-1, 5, 122))
        id_info_each = x_neg_each[:,:,-1] # patient id
        x_neg_each = x_neg_each[:,:,:120]

        for i in range(x_neg_each.shape[0]):
            # print(i)
            current_sub_feature = np.array([])
            current_sub_signal = x_neg_each[i, :, :]
            for j in range(21):
                windows_signal = current_sub_signal[:, j*5:(j*5+20)]
                win_feat = feature_extraction(windows_signal).T
                if len(current_sub_feature) == 0:
                    current_sub_feature = win_feat
                else:
                    current_sub_feature = np.hstack((current_sub_feature, win_feat))
            if len(all_subjects) == 0:
                all_subjects = current_sub_feature
            else:
                all_subjects = np.vstack((all_subjects, current_sub_feature))
        all_subjects_features = np.reshape(all_subjects, (x_neg_each.shape[0], 21, 31))

## Create negative samples  
categoried by four groups based on kmean

In [ ]:
if __name__ == '__main__':
    path = '/content/drive/My Drive/personal_files/deeptop/'
    file_path = ''
    file_path = path + 'data/stage_1/' + 'aa_all_stacked_neg.npz'
    x_neg = np.load(file_path)
    x_neg = x_neg['arr_0']
    for m in range(4):
        x_neg_each, id_each = [], []
        x_neg_each = x_neg
        all_subjects, id_info_each = np.array([]), np.array([])
        id_each = list(cluster_id.loc[cluster_id.flag == m+1, 'subject_id'])
        x_neg_each = x_neg_each[np.isin(x_neg_each[:, :, -1].astype(int), id_each),:].reshape((-1, 5, 122))
        id_info_each = x_neg_each[:,:,-1] # patient id
        x_neg_each = x_neg_each[:,:,:120]

        for i in range(x_neg_each.shape[0]):
            # print(i)
            current_sub_feature = np.array([])
            current_sub_signal = x_neg_each[i, :, :]
            for j in range(21):
                windows_signal = current_sub_signal[:, j*5:(j*5+20)]
                win_feat = feature_extraction(windows_signal).T
                if len(current_sub_feature) == 0:
                    current_sub_feature = win_feat
                else:
                    current_sub_feature = np.hstack((current_sub_feature, win_feat))
            if len(all_subjects) == 0:
                all_subjects = current_sub_feature
            else:
                all_subjects = np.vstack((all_subjects, current_sub_feature))
        all_subjects_features = np.reshape(all_subjects, (x_neg_each.shape[0], 21, 31))

        # add id info 
        id_info_each = np.repeat(id_info_each[:,0].reshape(id_info_each[:,0].shape[0],1), repeats=21, axis=1)
        id_info_each = id_info_each.reshape(id_info_each.shape[0], id_info_each.shape[1], 1)
        all_subjects_features = np.concatenate([all_subjects_features, id_info_each], -1) # last column: patient id
                
        path_new = ''
        path_new = path + 'data/stage_2/class_' + str(m+1) + '/' + file_path[72:-4] + 'slid_features.npz'
        print([m+1, all_subjects_features.shape])
        np.savez(path_new, all_subjects_features)

## Merge positive and negative samples without clustering

In [ ]:
path = '/content/drive/My Drive/personal_files/deeptop/data/stage_2/'
# positive
for i in range(4): # 0, 2, 4, 6
  data = np.array([])
  for j in range(4): # 1, 2, 3, 4
    data_each = np.array([])
    file_path = ''
    file_path = path + 'class_' + str(j+1) + '/ed_pos_' + str(2*i) + 'slid_features.npz'
    data_each = np.load(file_path)
    data_each = data_each['arr_0']
    if j==0:
      data = data_each
    else:
      data = np.concatenate((data, data_each), axis=0)
  print(data.shape)
  np.savez(path + 'ed_pos_' + str(i*2) + 'slid_features.npz', data)

In [ ]:
# negative
data = np.array([])
for i in range(4): # 1, 2, 3, 4  
  file_path = ''
  file_path = path + 'class_' + str(i+1) + '/ed_negslid_features.npz'
  data_each = np.array([])
  data_each = np.load(file_path)
  data_each = data_each['arr_0']
  if i == 0:
    data = data_each
  else:
    data = np.concatenate((data, data_each), axis=0)
print(data.shape)
np.savez(path + 'ed_negslid_features.npz', data)